In [ ]:

import re
import os
import keyword
import tempfile
import subprocess

import torch

from vllm import LLM, SamplingParams

In [ ]:
model_name = "Qwen/Qwen2.5-Math-1.5B-Instruct"

In [ ]:
model = LLM(model_name,
            trust_remote_code=True,
            dtype="bfloat16",
            max_num_seqs=8,
            max_model_len=4096,
            tensor_parallel_size=1,
            gpu_memory_utilization=0.8,
            seed=42
            )

In [ ]:
sampling_params = SamplingParams(
    temperature=0.7,              # randomness of the sampling
    min_p=0.01,
    top_p=0.8,
    skip_special_tokens=True,     # Whether to skip special tokens in the output.
    max_tokens=2400,
    # stop=["```\n"],
    include_stop_str_in_output=True,
)

In [ ]:
tokenizer = model.get_tokenizer()

In [ ]:
def create_first_msg(question):
    return [
            {"role": "system", "content": "Please integrate natural language reasoning with programs to solve the problem above, and, put your final answer within \\boxed{}."},
            {"role": "user", "content": question + "\n\nBegin your answer by importing sympy."}
        ]

In [ ]:
def generate_batch_message(messages):
    prompts = [
        tokenizer.apply_chat_template(
            message,
            tokenize=False,
            add_generation_prompt=True,
        )
        for message in messages
    ]

    outputs = model.generate(prompts=prompts, sampling_params=sampling_params)
    generations = [(prompt, generation.outputs[0].text) for prompt, generation in zip(prompts, outputs)]

    return generations

In [ ]:
def extract_python_code(text):
    pattern = r'```python\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    return "\n\n".join(matches)

def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return ""
    return matches[0]

In [ ]:
class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout

    def __call__(self, query):
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            
            try:
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
            except subprocess.TimeoutExpired:
                return False, f"Execution timed out after {self.timeout} seconds."

            stdout = result.stdout.strip()
            stderr = result.stderr.strip()

            if result.returncode == 0:
                return True, stdout
            else:
                # Process the error message to remove the temporary file path
                # This makes the error message cleaner and more user-friendly
                error_lines = stderr.split("\n")
                cleaned_errors = []
                for line in error_lines:
                    if temp_file_path in line:
                        # Remove the path from the error line
                        line = line.replace(temp_file_path, "<temporary_file>")
                    cleaned_errors.append(line)
                cleaned_error_msg = "\n".join(cleaned_errors)
                # Include stdout in the error case
                combined_output = f"{stdout}\n{cleaned_error_msg}" if stdout else cleaned_error_msg
                return False, combined_output

In [ ]:
messages = ["Triangle $ABC$ has side length $AB = 120$ and circumradius $R = 100$. Let $D$ be the foot of the perpendicular from $C$ to the line $AB$. What is the greatest possible length of segment $CD$?"] * 10

messages = [create_first_msg(message) for message in messages]

In [ ]:
gt = 180

In [ ]:
generations = generate_batch_message(messages)

In [ ]:
gold_generation = []

In [ ]:
for prompt, generation in generations:
    print(generation)
    python_code = extract_python_code(generation)
    try:
        boxed_out = int(extract_boxed_text(generation))
    except:
        boxed_out = extract_boxed_text(generation)
    is_success, output = PythonREPL()(python_code)
    try:
        output = int(output.split(".")[0]) if is_success else output
    except:
        output = output
    print("Python Output: {}".format(output))
    print("Boxed Text: {}".format(boxed_out))
    print("Boxed Out == Python Out: {}".format(boxed_out == output))
    print("===================")
    if (boxed_out == output) and (boxed_out == gt):
        gold_generation.append(generation)
        print("Gold Generation: {}".format(generation))


In [ ]:
gold_generation

# Gold Generation

1.
To solve the problem, we need to find the greatest possible length of segment \(CD\), where \(D\) is the foot of the perpendicular from \(C\) to the line \(AB\). Given that \(AB = 120\) and the circumradius \(R = 100\), we can use the properties of the circumcircle and the triangle to find the maximum length of \(CD\).

The length \(CD\) is maximized when \(C\) is at the highest possible position relative to \(AB\). This happens when \(C\) is on the line perpendicular to \(AB\) passing through the circumcenter \(O\), and \(C\) is on the opposite side of \(AB\) relative to \(O\).

The distance from the circumcenter \(O\) to the chord \(AB\) can be calculated using the formula:
\[ d = \sqrt{R^2 - \left(\frac{AB}{2}\right)^2} \]
where \(R\) is the circumradius and \(AB\) is the length of the chord.

Let's calculate \(d\):
\[ d = \sqrt{100^2 - \left(\frac{120}{2}\right)^2} = \sqrt{100^2 - 60^2} = \sqrt{10000 - 3600} = \sqrt{6400} = 80 \]

The maximum length of \(CD\) is then the sum of the circumradius \(R\) and the distance \(d\):
\[ CD = R + d = 100 + 80 = 180 \]

Let's confirm this with sympy:

```python
import sympy as sp

# Given values
AB = 120
R = 100

# Calculate the distance from the circumcenter to the chord AB
d = sp.sqrt(R**2 - (AB/2)**2)

# Calculate the maximum length of CD
CD_max = R + d

print(CD_max)
```
```output
180.000000000000
```
The greatest possible length of segment \(CD\) is \(\boxed{180}\).


2.